<a href="https://colab.research.google.com/github/shoboske/wine-quality-deep-learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install ucimlrepo scikit-learn pandas numpy matplotlib tensorflow

In [2]:
from ucimlrepo import fetch_ucirepo
import numpy as np
from sklearn.model_selection import train_test_split

# fetch dataset
wine_quality = fetch_ucirepo(id=186)
red_wine = wine_quality.data.original.query('color == "red"')
white_wine = wine_quality.data.original.query('color == "white"')

X_red = red_wine.drop(columns=['quality', 'color'])
y_red = red_wine['quality']

X_white = white_wine.drop(columns=['quality', 'color'])
y_white = white_wine['quality']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import tensorflow as tf

# Split data into training and testing sets
X_red_train, X_red_test, y_red_train, y_red_test = train_test_split(X_red, y_red, test_size=0.8, random_state=42)
X_white_train, X_white_test, y_white_train, y_white_test = train_test_split(X_white, y_white, test_size=0.8, random_state=42)

# Create a Pipeline for feature selection and scaling
# Define the pipeline steps
pipeline = Pipeline([
    ('selector', SelectKBest(f_classif, k=7)),  # Feature selection
    ('scaler', StandardScaler())  # Scaling
])

# Fit and transform the pipeline on the training data (for both red and white wine)
X_red_train_processed = pipeline.fit_transform(X_red_train, y_red_train)
X_white_train_processed = pipeline.fit_transform(X_white_train, y_white_train)

# Transform the test data (for both red and white wine)
X_red_test_processed = pipeline.transform(X_red_test)
X_white_test_processed = pipeline.transform(X_white_test)

# Red Wine Model
red_wine_model = tf.keras.models.Sequential(
    name="MLP - Red wine",
    [
        tf.keras.layers.Flatten(input_shape=X_red_train_processed.shape[1:]),
        tf.keras.layers.Dense(128, activation='relu'),  # Increased units
        tf.keras.layers.BatchNormalization(),  # Added Batch Normalization
        tf.keras.layers.Dropout(0.3),  # Adjusted dropout rate
        tf.keras.layers.Dense(64, activation='relu'),  # Decreased units, creating a bottleneck
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'), # Decreased units, creating a bottleneck
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(11, activation='softmax')  # Softmax in the output layer
    ]
)

# White Wine Model (similar structure, adjust input shape)
white_wine_model = tf.keras.models.Sequential(
    name="MLP - White wine",
    [
        tf.keras.layers.Flatten(input_shape=X_white_train_processed.shape[1:]),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(11, activation='softmax')
    ]
)

# Compile models (choose appropriate optimizer, loss, metrics)
red_wine_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
white_wine_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the models
red_wine_model.fit(X_red_train_processed, y_red_train, epochs=1000, validation_split=0.1, validation_freq=10, verbose=0)
white_wine_model.fit(X_white_train_processed, y_white_train, epochs=1000, validation_split=0.1, validation_freq=10, verbose=0)

In [27]:
# Evaluate the model (you can use various metrics like accuracy, precision, recall, etc.)
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd

# Predictions for red and white wine
y_red_wine_pred = red_wine_model.predict(X_red_test_processed)  # Predictions for red wine
y_white_wine_pred = white_wine_model.predict(X_white_test_processed) # Predictions for white wine

# Convert predictions to class labels (if necessary)
y_red_wine_pred_classes = np.argmax(y_red_wine_pred, axis=1)
y_white_wine_pred_classes = np.argmax(y_white_wine_pred, axis=1)

# Evaluate Red Wine Model
accuracy_red = accuracy_score(y_red_test, y_red_wine_pred_classes)
recall_red = recall_score(y_red_test, y_red_wine_pred_classes, average='weighted')
precision_red = precision_score(y_red_test, y_red_wine_pred_classes, average='weighted', zero_division=0.0)
f1_red = f1_score(y_red_test, y_red_wine_pred_classes, average='weighted')

# Evaluate White Wine Model
accuracy_white = accuracy_score(y_white_test, y_white_wine_pred_classes)
recall_white = recall_score(y_white_test, y_white_wine_pred_classes, average='weighted')
precision_white = precision_score(y_white_test, y_white_wine_pred_classes, average='weighted', zero_division=0.0)
f1_white = f1_score(y_white_test, y_white_wine_pred_classes, average='weighted')

# Create data for DataFrame
data = [
    [f"{red_wine_model.name} ({', '.join(map(str, [*red_wine_model.layers[0].input.shape[1:], *[layer.units for layer in red_wine_model.layers if isinstance(layer, tf.keras.layers.Dense)]]))})", accuracy_red, recall_red, precision_red, f1_red],
    [f"{white_wine_model.name} ({', '.join(map(str, [*white_wine_model.layers[0].input.shape[1:], *[layer.units for layer in white_wine_model.layers if isinstance(layer, tf.keras.layers.Dense)]]))})", accuracy_white, recall_white, precision_white, f1_white]
]

headers = ["Model", "Accuracy", "Recall", "Precision", "F1 Score"]

print(red_wine_model.summary())
print(white_wine_model.summary())

print(pd.DataFrame(data, None, headers))

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
                                           Model  Accuracy    Recall  \
0                                  MLP (100,200)  0.474219  0.474219   
1                                       MLP Adam  0.549219  0.549219   
2  sequential_5 (11, 80, 80, 80, 80, 80, 80, 11)  0.546094  0.546094   

   Precision  F1 Score  
0   0.396354  0.430062  
1   0.453560  0.496695  
2   0.523485  0.533358  
